In [1]:
using Statistics
using CairoMakie
using Integrals
using Printf
using JLD2

include("l1cheby.jl")
@load "coefs.jld2" coefs1 coefs2 coefs3;

In [2]:
function L₁(A::Float64, B::Float64, logH::Float64)
    H = 10^logH

    f(u) = (u + H) / (u - H - (u + H)*exp(-2u))
    g(u) = (exp(-u*(2+B)) + exp(-u*(2-B))) * cos(u*A)
    h(u, _) = (f(u) * g(u) + exp(-u)) / u
    
    paths = [(0.0, H+im), (H+im, H+1.0), (H+1.0, Inf)]
    M = 0.0
    for path in paths
        IP = IntegralProblem(h, path)
        M += solve(IP, QuadGKJL(); reltol=1e-16, abstol=1e-16).u
    end
    real(M)
end;

In [3]:
Amin, Amax = 0.0, 0.5
Bmin, Bmax = 0.0, 1.0
logHmin, logHmax = -2.0, 2.0;

In [4]:
function plot_max_abs_val_coefs(domain::Int; savefig::Bool = false)
    if domain == 1
        coefs = coefs1
    elseif domain == 2
        coefs = coefs2
    elseif domain == 3
        coefs = coefs3
    else
        throw(DomainError(domain, "Invalid domain value"))
    end
    
    coefs_A = vec(maximum(abs, coefs; dims=(2, 3)))
    coefs_B = vec(maximum(abs, coefs; dims=(1, 3)))
    coefs_logH = vec(maximum(abs, coefs; dims=(1, 2)))
    
    fig = Figure(size=(500, 350))
    ax = Axis(
        fig[1, 1],
        yscale=log10,
        title="Maximum absolute value of coefficients in domain $domain",
        xlabel="Coefficient order",
    )
    lines!(ax, 0:size(coefs, 1)-1, coefs_A, color=:red, linestyle=:dash, label="A")
    lines!(ax, 0:size(coefs, 2)-1, coefs_B, color=:green, linestyle=:dot, label="B")
    lines!(ax, 0:size(coefs, 3)-1, coefs_logH, color=:blue, linestyle=:solid, label="log(H)")
    axislegend(ax)
    savefig && save("max_abs_coefs$domain.svg", fig)
    fig
end;

In [5]:
plot_max_abs_val_coefs.(1:3, savefig=true);

In [6]:
# nptsA, nptsB, nptslogH = 30, 40, 80

# Av = LinRange(Amin, Amax, nptsA)
# Bv = LinRange(Bmin, Bmax, nptsB)
# logHv = LinRange(logHmin, logHmax, nptslogH)

# L1v = zeros(nptsA, nptsB, nptslogH)
# L1cbv = zeros(nptsA, nptsB, nptslogH)

# t1 = time()
# for (i, A) in enumerate(Av), (j, B) in enumerate(Bv), (k, logH) in enumerate(logHv)
#     L1v[i, j, k] = L₁(A, B, logH)
# end
# t2 = (time() - t1) / (nptsA * nptsB * nptslogH)

# t1cb = time()
# for (i, A) in enumerate(Av), (j, B) in enumerate(Bv), (k, logH) in enumerate(logHv)
#     L1cbv[i, j, k] = L₁_cheby(A, B, logH)
# end
# t2cb = (time() - t1cb) / (nptsA * nptsB * nptslogH)

# abserr = abs.(L1v - L1cbv)

# @save "l1v.jld2" L1v
# @save "l1cbv.jld2" L1cbv
# @save "abserr.jld2" abserr

# @printf("The approximation is, on average, %.2f times faster than the integral expression.", t2/t2cb)

@load "abserr.jld2" abserr;

In [7]:
abserr_mean = mean(abserr)
abserr_std = std(abserr)
abserr_pct = sum(abserr .< 1e-14) / length(abserr) * 100

result = """
The residual data has a mean absolute value of $(@sprintf("%.2e", abserr_mean)),
with a standard deviation of $(@sprintf("%.2e", abserr_std)),
and $(@sprintf("%.2f", abserr_pct))% of the residues smaller than 1e-14.
"""
println(result)

The residual data has a mean absolute value of 1.07e-15,
with a standard deviation of 8.78e-16,
and 99.99% of the residues smaller than 1e-14.

